In [ ]:
# Import necessary modules of astroCAST
from astroCAST.denoising import SubFrameGenerator, Network
from astroCAST.analysis import Video
from pathlib import Path

In [ ]:
folder = Path("/media/janrei1/data/method_testing/")

input_file = folder.joinpath("22A7x9-1.from_single_file.h5") # your input file name here. 'h5' or 'tdb' is recommended for parallel processing 
assert input_file.is_file()

model_path = folder.joinpath("model")
infer_path = input_file.with_suffix(".denoised.tiff")

# Train model

In [ ]:
param = dict(paths=input_file, loc="mc/ch0", input_size=(128, 128), pre_post_frame=5, gap_frames=0,
                 normalize="global", cache_results=True, in_memory=True)

train_gen = SubFrameGenerator(padding=None, batch_size=25, max_per_file=50,
                               allowed_rotation=[1, 2, 3], allowed_flip=[0, 1], shuffle=True, **param)

val_gen = SubFrameGenerator(padding=None, batch_size=25, max_per_file=5,
                                   allowed_rotation=[0], allowed_flip=[-1], shuffle=True, **param)


net = Network(train_generator=train_gen, val_generator=val_gen, n_stacks=1, kernel=4, batchNormalize=False, use_cpu=True)
net.run(batch_size=train_gen.batch_size, num_epochs=5, patience=2, min_delta=0.01, save_model=model_path, load_weights=False)


# Denoise

In [ ]:
inf_gen = SubFrameGenerator(padding="edge", batch_size=200, allowed_rotation=[0], allowed_flip=[-1],
                                shuffle=False, **param)

# TODO: proper way to load model

inf_gen.infer(model=net.model, output=infer_path.as_posix(), rescale=False)

In [ ]:
"done"